In [6]:
# import modules
import xml.etree.ElementTree as ET
import pandas as pd
import sys
import string
import re
import nltk
import sys
import os
import gensim
import logging

# Set up basic configuration for logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]  # Logs to the console
)

# Add the src directory to sys.path
sys.path.append(os.path.abspath(r'C:\dev\NLP2RE_Sandbox\src'))


print(sys.version)
print(sys.executable)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
c:\ProgramData\anaconda3\python.exe


In [2]:
# Specify the path to your XML file and namespace
xml_file_path = r'C:\dev\NLP2RE_Sandbox\data\work_data\2007-ertms.xml'
namespace = {'ns': 'req_document.xsd'}

In [3]:
# Parse .xml to df
from utils import parse_xml

# import utils.ParseXML as ParseXML
df = parse_xml.process_xml_with_namespace(xml_file_path, namespace)
df.tail(10)

,tag,text,id,path
621,meaning,Transmission of ETCS information from a train ...,,req_document/p/glossary/glossary_item/meaning
622,term,Train trip,,req_document/p/glossary/glossary_item/term
623,meaning,"Is used when a train passes a ""danger"" signal,...",,req_document/p/glossary/glossary_item/meaning
624,term,Warning,,req_document/p/glossary/glossary_item/term
625,meaning,Audible and/or visual indication to alert the ...,,req_document/p/glossary/glossary_item/meaning
626,term,Wheelslip,,req_document/p/glossary/glossary_item/term
627,meaning,When a traction-driven wheel loses adhesion wi...,,req_document/p/glossary/glossary_item/meaning
628,term,Wheelslide,,req_document/p/glossary/glossary_item/term
629,meaning,When a braked wheel loses adhesion with the ra...,,req_document/p/glossary/glossary_item/meaning
630,title,Other technical functions,11,req_document/p/title


In [5]:
# Create 'text_clean' attribute in df (list of tokens)
from utils import clean_data

df['text_clean'] = df['text'].apply(lambda x: clean_data.data_preparation(x))

df.head()

NameError: name 'logging' is not defined

In [ ]:
 # Create a Dictionary: Associate each word in the corpus with a unique integer ID

from gensim import corpora

dct = corpora.Dictionary(df['text_clean'])
print(dct.token2id)



{'ertm': 0, 'etc': 1, 'fr': 2, 'function': 3, 'requir': 4, 'specif': 5, '00': 6, '5': 7, '06': 8, '2007': 9, '21': 10, '003204': 11, 'era': 12, 'introduct': 13, '': 14, '0': 15, '2': 16, '3': 17, 'a': 18, 'all': 19, 'and': 20, 'been': 21, 'consist': 22, 'contain': 23, 'control': 24, 'defin': 25, 'document': 26, 'european': 27, 'few': 28, 'for': 29, 'from': 30, 'have': 31, 'implement': 32, 'in': 33, 'manag': 34, 'not': 35, 'onli': 36, 'oper': 37, 'primarili': 38, 'rail': 39, 'reason': 40, 'remov': 41, 'sr': 42, 'system': 43, 'technic': 44, 'term': 45, 'the': 46, 'therefor': 47, 'thi': 48, 'traffic': 49, 'train': 50, 'version': 51, 'applic': 52, 'be': 53, 'cc': 54, 'condit': 55, 'everi': 56, 'if': 57, 'is': 58, 'it': 59, 'level': 60, 'lower': 61, 'm': 62, 'mandatori': 63, 'may': 64, 'note': 65, 'o': 66, 'of': 67, 'option': 68, 'respect': 69, 'safeti': 70, 'shall': 71, 'state': 72, 'that': 73, 'to': 74, 'tsi': 75, 'where': 76, 'gener': 77, 'basic': 78, 'allow': 79, 'drive': 80, 'driver': 

In [ ]:
# Return a list of the n most common words and their counts from the most common to the least.
print(dct.most_common(10))

[('the', 548), ('train', 250), ('', 231), ('to', 214), ('shall', 210), ('be', 189), ('a', 157), ('of', 149), ('etc', 104), ('and', 97)]


## Corner cases:
joint → _joint 
jointed → _joint, ed 
disjointed → _di, s, jo, int, ed 
unisex → _un, ise, x 
true → _true 
untrue → _un, tr, ue 
estimate → _estimate 
overestimate → _over, est, imate

### Syntax
Below is the syntax for the gensim.utils.tokenize() function:

```python
gensim.utils.tokenize(text, lowercase=True, deacc=False, errors='strict', to_lower=False, lower=False)
```

text is the input text to be tokenized.

lowercase is an optional parameter that specifies whether to convert the text to lowercase before tokenization. The default value is True.

deacc is an optional parameter specifying whether to remove text accent marks. The default value is False.

errors is an optional parameter that specifies how to handle decoding errors in the text. The default value is 'strict'.

to_lower and lower are both optional parameters that are the same as lowercase and are used as a convenient alias.

In [ ]:
from gensim.utils import tokenize

text = """ Welcome to Educative Answers.
        joint
        jointed 
        disjointed 
        unisex 
        true 
        untrue 
        estimate 
        overestimate 
        """

tokens = list(tokenize(text,lowercase=True))
print(tokens)


['welcome', 'to', 'educative', 'answers', 'joint', 'jointed', 'disjointed', 'unisex', 'true', 'untrue', 'estimate', 'overestimate']


In [ ]:
import nltk
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))
stopped_tokens = [word for word in tokens if word not in en_stop]

## Remove costum stopwords from a dictionary

In [ ]:
# Initialize "stopword" list
from gensim.parsing.preprocessing import STOPWORDS

stopwords = set(STOPWORDS)
stopwords.add("ertm")
stopwords.add("fr")
stopwords.add("shall")
stopwords.add("")

print(stopwords)

{'ten', '', 'nevertheless', 'get', 'toward', 'therein', 'did', 'call', 'became', 'doesn', 'latter', 'can', 'up', 'co', 'seems', 'interest', 'along', 'mill', 'if', 'our', 'you', 'my', 'which', 'where', 'thick', 'whom', 'whereupon', 'well', 'neither', 'would', 'whereafter', 'herself', 'former', 'same', 'do', 'should', 'too', 'sometime', 'further', 'whereas', 'moreover', 'un', 'noone', 'via', 'elsewhere', 'whence', 'except', 'give', 'me', 'almost', 'twelve', 'yours', 'being', 'ever', 'formerly', 'afterwards', 'something', 'or', 'their', 'above', 'always', 'full', 'nothing', 'is', 'hence', 'side', 'upon', 'ours', 'the', 'beyond', 'beside', 'not', 'those', 'three', 'etc', 'must', 'part', 'had', 'an', 'may', 'as', 'back', 'after', 'each', 'wherein', 'thru', 'at', 'does', 'off', 'someone', 'however', 'beforehand', 'couldnt', 'that', 'least', 'see', 'hundred', 'eleven', 'itself', 'front', 'than', 'really', 'it', 'fr', 'most', 'was', 'between', 'why', 'on', 'although', 'into', 'empty', 'towards

In [ ]:
# Remove the words from the dictionary

# Get the ids of the words to be removed
ids_to_remove = [dct.token2id[word] for word in stopwords if word in dct.token2id]

# Filter the dictionary
dct.filter_tokens(bad_ids=ids_to_remove)

# Compact the dictionary to remove gaps in the id sequence
dct.compactify()

# Print the remaining words in the dictionary
print(dct.token2id)

{'function': 0, 'requir': 1, 'specif': 2, '00': 3, '5': 4, '06': 5, '2007': 6, '21': 7, '003204': 8, 'era': 9, 'introduct': 10, '0': 11, '2': 12, '3': 13, 'consist': 14, 'contain': 15, 'control': 16, 'defin': 17, 'document': 18, 'european': 19, 'implement': 20, 'manag': 21, 'onli': 22, 'oper': 23, 'primarili': 24, 'rail': 25, 'reason': 26, 'remov': 27, 'sr': 28, 'technic': 29, 'term': 30, 'therefor': 31, 'thi': 32, 'traffic': 33, 'train': 34, 'version': 35, 'applic': 36, 'cc': 37, 'condit': 38, 'everi': 39, 'level': 40, 'lower': 41, 'm': 42, 'mandatori': 43, 'note': 44, 'o': 45, 'option': 46, 'respect': 47, 'safeti': 48, 'state': 49, 'tsi': 50, 'gener': 51, 'basic': 52, 'allow': 53, 'drive': 54, 'driver': 55, 'inform': 56, 'provid': 57, 'safe': 58, 'abl': 59, 'movement': 60, 'shunt': 61, 'supervis': 62, 'area': 63, 'authoris': 64, 'perform': 65, 'possibl': 66, 'prevent': 67, 'rbc': 68, 'traction': 69, 'unit': 70, '500': 71, 'h': 72, 'maximum': 73, 'speed': 74, '1': 75, 'activ': 76, 'an

In [ ]:
# Return a list of the n most common words and their counts from the most common to the least.
dct.compactify()
print("Number of unique tokens:", len(dct))
print(dct.most_common(10))

Number of unique tokens: 559
[('train', 250), ('driver', 90), ('speed', 78), ('oper', 64), ('equip', 60), ('data', 60), ('inform', 53), ('movement', 53), ('brake', 52), ('supervis', 51)]


In [ ]:
dct.filter_extremes(no_below=0, no_above=0.075, keep_n=1000000)
# Return a list of the n most common words and their counts from the most common to the least.
dct.compactify()
print("Number of unique tokens:", len(dct))
print(dct.most_common(10))

Number of unique tokens: 547
[('function', 37), ('nation', 33), ('requir', 32), ('shunt', 32), ('level', 30), ('dmi', 30), ('trainborn', 30), ('tracksid', 29), ('transmiss', 29), ('valu', 29)]


# Functions

In [ ]:
from utils import stpwrds

# Define some new stopwords to add
new_stopwords = ['example', 'string', 'some', 'stopwords']

# Add the new stopwords to the JSON file
stpwrds.add_custom_stopwords_to_json(new_stopwords)

# Define an input string to clean
input_string = "This is an example of a string with some stopwords that need to be removed."

# Remove stopwords from the input string
cleaned_string = stpwrds.remove_custom_stopwords_from_string(input_string)

print("Original String:")
print(input_string)
print("\nCleaned String:")
print(cleaned_string)
